In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',encoding = "ISO-8859-1")
df

In [ ]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis = 1, inplace = True)
df

In [ ]:
df.groupby('v1').v1.count().plot.bar(ylim = 0)
plt.show()

In [ ]:
dic = {'ham':1, 'spam':0}
df.v1 = df.v1.map(dic)
df

In [ ]:
df['processedtext'] = df['v2']
df

In [ ]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import warnings
warnings.filterwarnings('ignore')

stemmer = PorterStemmer()
words = stopwords.words("english")

df['processedtext'] = df['processedtext'].apply(lambda x: ' '.join([stemmer.stem(i)
                            for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [ ]:
df['processedtext'] = df['processedtext'].str.lower()

df['processedtext'] = df['processedtext'].str.replace("[^a-zA-Z#]", " ")

df['processedtext'] = df['processedtext'].apply(lambda x: ' '.join([w for w in 
                                                                   x.split() if len(w)>3]))

In [ ]:
y = df['v1']
x = df['processedtext']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tfidf = TfidfVectorizer(stop_words ='english' , max_df = 0.7)
df_tfIdf = vectorizer_tfidf.fit_transform(x.values.astype('U'))
print(vectorizer_tfidf.get_feature_names()[:10])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_tfIdf, y, test_size = 0.10, 
                                                  random_state =1, shuffle = True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier

model = PassiveAggressiveClassifier(max_iter = 1000, random_state = 1, tol = 1e-3).fit(X_train, y_train)
print(model.score(X_train, y_train))

In [ ]:
y_pred = model.predict(X_val)
print(model.score(X_val, y_val))

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_val, y_pred))

In [ ]:
from sklearn.decomposition import TruncatedSVD
TruncatedSVD(y_val, y_pred)